In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchsummary import summary

import matplotlib.pyplot as plt

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
training_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [6]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [7]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [8]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" # it is mps which is gpu but not cuda for mac
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using mps device


In [9]:
class NN(nn.Module): # nn.Module is base class for all models in pytorch
    def __init__(self): # constructor of class
        super().__init__() # super() is used to call the constructor of parent class
        self.flatten  = nn.Flatten() # flatten layer
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
# Create an instance of the model, including the device it is on.
model = NN().to(device) # to() is used to move the model to the device
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
print(model)


NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [10]:
# define the training loop
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset) # get the size of the dataset
    model.train() # set the model to training mode
    for batch, (X, y) in enumerate(dataloader): # iterate over the data, batch is the index of the batch, X is the input and y is the output
        X, y = X.to(device), y.to(device) # move the data to the device
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        # Backpropagation
        loss.backward() # compute the gradient of the loss with respect to the model parameters
        optimizer.step() # adjust the parameters by the gradients collected in the backward pass
        optimizer.zero_grad() # reset the gradients to zero
        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X) 
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval() # set the model to evaluation mode
    test_loss, correct = 0, 0
    with torch.no_grad(): # disable gradient computation
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item() # sum up the loss
            correct += (pred.argmax(1) == y).type(torch.float).sum().item() # get the index of the max log-probability
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [12]:
epochs = 5 # number of epochs
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.305561  [   64/60000]
loss: 2.289518  [ 6464/60000]
loss: 2.270289  [12864/60000]
loss: 2.263437  [19264/60000]
loss: 2.246391  [25664/60000]
loss: 2.215353  [32064/60000]
loss: 2.225014  [38464/60000]
loss: 2.188428  [44864/60000]
loss: 2.180696  [51264/60000]
loss: 2.147310  [57664/60000]
Test Error: 
 Accuracy: 48.1%, Avg loss: 2.141722 

Epoch 2
-------------------------------
loss: 2.155007  [   64/60000]
loss: 2.141639  [ 6464/60000]
loss: 2.078445  [12864/60000]
loss: 2.096678  [19264/60000]
loss: 2.038934  [25664/60000]
loss: 1.970310  [32064/60000]
loss: 2.010129  [38464/60000]
loss: 1.921651  [44864/60000]
loss: 1.928320  [51264/60000]
loss: 1.852824  [57664/60000]
Test Error: 
 Accuracy: 52.1%, Avg loss: 1.853042 

Epoch 3
-------------------------------
loss: 1.890126  [   64/60000]
loss: 1.856437  [ 6464/60000]
loss: 1.739494  [12864/60000]
loss: 1.787724  [19264/60000]
loss: 1.668804  [25664/60000]
loss: 1.621364  [32064/600

In [13]:
# check whether the model is on the device
device = next(model.parameters()).device
print(f"Model is on device: {device}")

Model is on device: mps:0


In [14]:
# Temporarily move the model to CPU for summary
model_cpu = model.to("cpu")
# Run summary on the CPU (since torchsummary only supports 'cpu' or 'cuda')
summary(model_cpu, input_size=(1, 28, 28), device="cpu")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


In [15]:
# print the total number of parameters and trainable parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")

Total parameters: 669706
Trainable parameters: 669706


In [17]:
# summarize the model parameters
for name, param in model.named_parameters():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param}")
    print(f"Parameter shape: {param.shape}")
    print(f"Requires gradient: {param.requires_grad}")
    print("-" * 50)

Parameter name: linear_relu_stack.0.weight
Parameter value: Parameter containing:
tensor([[ 0.0111, -0.0163,  0.0044,  ..., -0.0006,  0.0274,  0.0065],
        [-0.0118,  0.0021,  0.0163,  ...,  0.0021, -0.0304,  0.0215],
        [ 0.0244,  0.0233, -0.0097,  ..., -0.0152,  0.0350, -0.0285],
        ...,
        [ 0.0047, -0.0119, -0.0056,  ..., -0.0351,  0.0062,  0.0234],
        [-0.0012,  0.0035, -0.0316,  ..., -0.0269,  0.0182,  0.0060],
        [ 0.0156, -0.0253, -0.0042,  ..., -0.0286, -0.0180, -0.0263]],
       requires_grad=True)
Parameter shape: torch.Size([512, 784])
Requires gradient: True
--------------------------------------------------
Parameter name: linear_relu_stack.0.bias
Parameter value: Parameter containing:
tensor([ 3.1299e-02, -2.4866e-02, -2.4687e-03,  3.3257e-02, -1.2573e-02,
        -2.9465e-02,  3.6867e-02, -1.1514e-02, -1.7470e-03,  8.7670e-03,
        -2.8088e-02, -2.8737e-02, -3.4039e-02, -1.5097e-02, -3.3807e-02,
        -4.1252e-03, -1.3162e-02,  3.1557e-

In [16]:
model.named_parameters

<bound method Module.named_parameters of NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)>

In [18]:
model.state_dict()

OrderedDict([('linear_relu_stack.0.weight',
              tensor([[ 0.0111, -0.0163,  0.0044,  ..., -0.0006,  0.0274,  0.0065],
                      [-0.0118,  0.0021,  0.0163,  ...,  0.0021, -0.0304,  0.0215],
                      [ 0.0244,  0.0233, -0.0097,  ..., -0.0152,  0.0350, -0.0285],
                      ...,
                      [ 0.0047, -0.0119, -0.0056,  ..., -0.0351,  0.0062,  0.0234],
                      [-0.0012,  0.0035, -0.0316,  ..., -0.0269,  0.0182,  0.0060],
                      [ 0.0156, -0.0253, -0.0042,  ..., -0.0286, -0.0180, -0.0263]])),
             ('linear_relu_stack.0.bias',
              tensor([ 3.1299e-02, -2.4866e-02, -2.4687e-03,  3.3257e-02, -1.2573e-02,
                      -2.9465e-02,  3.6867e-02, -1.1514e-02, -1.7470e-03,  8.7670e-03,
                      -2.8088e-02, -2.8737e-02, -3.4039e-02, -1.5097e-02, -3.3807e-02,
                      -4.1252e-03, -1.3162e-02,  3.1557e-02, -2.3931e-02, -9.9615e-03,
                       1.5670e-0

In [19]:
# print state_dict 
for name, param in model.state_dict().items():
    print(f"Parameter name: {name}")
    print(f"Parameter value: {param}")
    print(f"Parameter shape: {param.shape}")
    print("-" * 50)


Parameter name: linear_relu_stack.0.weight
Parameter value: tensor([[ 0.0111, -0.0163,  0.0044,  ..., -0.0006,  0.0274,  0.0065],
        [-0.0118,  0.0021,  0.0163,  ...,  0.0021, -0.0304,  0.0215],
        [ 0.0244,  0.0233, -0.0097,  ..., -0.0152,  0.0350, -0.0285],
        ...,
        [ 0.0047, -0.0119, -0.0056,  ..., -0.0351,  0.0062,  0.0234],
        [-0.0012,  0.0035, -0.0316,  ..., -0.0269,  0.0182,  0.0060],
        [ 0.0156, -0.0253, -0.0042,  ..., -0.0286, -0.0180, -0.0263]])
Parameter shape: torch.Size([512, 784])
--------------------------------------------------
Parameter name: linear_relu_stack.0.bias
Parameter value: tensor([ 3.1299e-02, -2.4866e-02, -2.4687e-03,  3.3257e-02, -1.2573e-02,
        -2.9465e-02,  3.6867e-02, -1.1514e-02, -1.7470e-03,  8.7670e-03,
        -2.8088e-02, -2.8737e-02, -3.4039e-02, -1.5097e-02, -3.3807e-02,
        -4.1252e-03, -1.3162e-02,  3.1557e-02, -2.3931e-02, -9.9615e-03,
         1.5670e-02, -1.0803e-03, -1.2151e-02, -2.0947e-02, -1.88

In [24]:
# check the first layer weight
model.state_dict()["linear_relu_stack.0.weight"]

tensor([[ 0.0111, -0.0163,  0.0044,  ..., -0.0006,  0.0274,  0.0065],
        [-0.0118,  0.0021,  0.0163,  ...,  0.0021, -0.0304,  0.0215],
        [ 0.0244,  0.0233, -0.0097,  ..., -0.0152,  0.0350, -0.0285],
        ...,
        [ 0.0047, -0.0119, -0.0056,  ..., -0.0351,  0.0062,  0.0234],
        [-0.0012,  0.0035, -0.0316,  ..., -0.0269,  0.0182,  0.0060],
        [ 0.0156, -0.0253, -0.0042,  ..., -0.0286, -0.0180, -0.0263]])

In [25]:
# Access the weights and bias of the first layer
print(model.linear_relu_stack)
# Access the first layer
fc1_weights = model.linear_relu_stack[0].weight
fc1_bias = model.linear_relu_stack[0].bias
# Print the weights and bias
print("Weights of fc1 layer:", fc1_weights)
print("Bias of fc1 layer:", fc1_bias)


Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Linear(in_features=512, out_features=512, bias=True)
  (3): ReLU()
  (4): Linear(in_features=512, out_features=10, bias=True)
)
Weights of fc1 layer: Parameter containing:
tensor([[ 0.0111, -0.0163,  0.0044,  ..., -0.0006,  0.0274,  0.0065],
        [-0.0118,  0.0021,  0.0163,  ...,  0.0021, -0.0304,  0.0215],
        [ 0.0244,  0.0233, -0.0097,  ..., -0.0152,  0.0350, -0.0285],
        ...,
        [ 0.0047, -0.0119, -0.0056,  ..., -0.0351,  0.0062,  0.0234],
        [-0.0012,  0.0035, -0.0316,  ..., -0.0269,  0.0182,  0.0060],
        [ 0.0156, -0.0253, -0.0042,  ..., -0.0286, -0.0180, -0.0263]],
       requires_grad=True)
Bias of fc1 layer: Parameter containing:
tensor([ 3.1299e-02, -2.4866e-02, -2.4687e-03,  3.3257e-02, -1.2573e-02,
        -2.9465e-02,  3.6867e-02, -1.1514e-02, -1.7470e-03,  8.7670e-03,
        -2.8088e-02, -2.8737e-02, -3.4039e-02, -1.5097e-02, -3.3807e-02,
        -4.125

In [21]:
# save model state_dict
# only the model's state_dict is saved
torch.save(model.state_dict(), "model.pth")
# load model state_dict
# model must be created before loading the state_dict
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten  = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28,512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512,10),
        )
    def forward(self,x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model2 = NN()


In [22]:
model2.named_parameters

<bound method Module.named_parameters of NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)>

In [23]:
model2.load_state_dict(torch.load("model.pth"))
# model2 is now loaded with the saved state_dict, but it is not trained
model2.eval()

/var/folders/g0/phgwvz7x3sz18gfzz4zv9rq40000gn/T/ipykernel_72634/309678342.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model2.load_state_dict(torch.load("model.pth")

NN(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)